The code imports several libraries and modules that will be used in the code. Here is a line-by-line explanation:

import pandas as pd: This imports the Pandas library and assigns it the alias pd. Pandas is a powerful library used for data manipulation and analysis.

import requests: This imports the Requests library, which allows sending HTTP requests in Python.

import xlsxwriter: This imports the XlsxWriter library, which is used to create Excel files.

import math: This imports the math module, which provides mathematical functions and constants.

from bs4 import BeautifulSoup: This imports the BeautifulSoup class from the BeautifulSoup library. BeautifulSoup is used for web scraping and parsing HTML and XML documents.

In [76]:
import pandas as pd
import requests 
import xlsxwriter 
import math
from bs4 import BeautifulSoup
import csv
import numpy as np
import time

The code retrieves data from a website and extracts information from it. Let's go through it step by step:

The code starts by importing necessary libraries such as requests, BeautifulSoup, numpy, pandas, xlsxwriter, and csv. These libraries will be used for making HTTP requests, parsing HTML, and working with data in various formats.

The code then assigns a URL to the variable URL. In this case, the URL is 'https://dhan.co/nifty-stocks-list/nifty-50/'. This is the website from which the data will be retrieved.

The code uses a try-except block to handle any exceptions that may occur during the execution of the code. It attempts to make a request to the specified URL using the requests.get() function and assigns the response to the variable r. If an exception occurs, it will be caught and an error message will be printed.

If the request is successful, the code creates a BeautifulSoup object called soup by passing the HTML content of the response (r.text) and the parser to be used ('lxml').

The code finds an element in the HTML with the class "d-flex flex-wrap pl-4 mt-4" using the find() method on the soup object and assigns it to the variable table. This element contains the data we are interested in.

The code initializes an empty list called data.

It then loops through each element with the class "col-md-6 col-12 pl-0" within the table object using the find_all() method. This class represents individual rows in the table.

For each row, the code extracts the text of the link (row.a.text) and removes any leading or trailing whitespace using the strip() method. The extracted text represents the name of a company.

The extracted name is then appended to the data list.

In [28]:
URL = 'https://dhan.co/nifty-stocks-list/nifty-50/'
try:
    r = requests.get(URL)
    # Rest of the code
except requests.RequestException as e:
    print("Error occurred:", e)
    # Handle the error gracefully
soup = BeautifulSoup(r.text,'lxml')
table = soup.find(class_="d-flex flex-wrap pl-4 mt-4")
data = []
for row in table.find_all(class_ = "col-md-6 col-12 pl-0"):
    name = row.a.text.strip()
    data.append(name)

The code opens a file named "nifty_50.csv" in write mode using the open() function and assigns it to the variable f. The newline='' argument is provided to ensure that no extra newline characters are added to the file.

A csv.writer object is created using the file object f. This object, assigned to the variable writer, will be used to write data to the CSV file.

The code writes a single row to the CSV file using the writerow() method of the writer object. This row contains a single column with the header name "Company_Name". This serves as the header row in the CSV file.

The code then enters a loop that iterates over each element in the data list obtained from the previous code.

For each element in data, the code performs the following steps:

It removes any commas (,) from the current element by splitting the string at commas and joining the resulting substrings together using join(). The result is assigned to the variable company_name.
Any leading or trailing whitespace is removed from company_name using the strip() method.
The cleaned company_name is written as a single-row list to the CSV file using the writerow() method. This creates a new row in the CSV file with the company name in the "Company_Name" column.

In [45]:
with open('nifty_50.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['Company_Name'])  # Writing the header row

    for row in data:
        company_name = ''.join(row.split(',')).strip()  # Removing commas and stripping whitespace
        writer.writerow([company_name])  # Writing each row with the updated company name

In [ ]:
stocks = pd.read_csv('S&P500.csv')
stocks

In [33]:
algo_key ="Use-your-api-key"
symbol = "IBM"
url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={algo_key}'
url1 = f'https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={symbol}&apikey={algo_key}'
r = requests.get(url)
r1 = requests.get(url1)
data = r.json()
data1 = r1.json()

{'Global Quote': {'01. symbol': 'IBM',
  '02. open': '121.4100',
  '03. high': '122.8600',
  '04. low': '121.1100',
  '05. price': '122.8400',
  '06. volume': '4564825',
  '07. latest trading day': '2023-05-12',
  '08. previous close': '120.9000',
  '09. change': '1.9400',
  '10. change percent': '1.6046%'}}

The following code performs individual API calls to retrieve stock data for multiple symbols from the Alpha Vantage API. Here's a line-by-line breakdown of the code:

symbol = [i for i in stocks['Ticker']]: This line creates a list symbol that contains all the ticker symbols from the 'Ticker' column of the stocks DataFrame.

col = ["Ticker", "Stock_Price", "Market_Cap", "Number_of_shares_to_buy"]: This line defines a list col that contains the column names for the data to be collected.

df2 = []: This line initializes an empty list df2 which will store the collected data as DataFrames.

num_iterations = min(20, 500 // 5): This line calculates the number of iterations required to stay within the rate limits. It takes the minimum value between 20 (a limit set by the code) and 500 divided by 5 (since the rate limit is 5 requests per minute).

In [ ]:
## Batch api calling is not available in Alphavantage, so we are calling 500 apis individually

symbol = [i for i in stocks['Ticker']]
col = ["Ticker", "Stock_Price", "Market_Cap", "Number_of_shares_to_buy"]
df2 = []

# Calculate the number of iterations required to stay within the rate limits
num_iterations = min(20, 500 // 5)  # Limiting to 20 symbols or maximum allowed by the daily limit

for i in range(num_iterations):
    stock = symbol[i]
    try:
        url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={stock}&apikey={algo_key}'
        url1 = f'https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={stock}&apikey={algo_key}'
        data = requests.get(url).json()
        data1 = requests.get(url1).json()
        df = pd.DataFrame([[stock, data1['Global Quote']['05. price'], data['MarketCapitalization'], None]], columns=col)
        df2.append(df)
    except Exception as e:
        print(f"An error occurred for stock {stock}: {e}")

    # Add a delay to stay within the rate limit of 5 requests per minute
    if i < num_iterations - 1:
        time.sleep(12)  # Delay for 12 seconds between iterations

result_df = pd.concat(df2, ignore_index=True)

The code is responsible for calculating the number of shares to buy for each stock in a portfolio based on the portfolio size and the stock's current price.

Here's a breakdown of the code:

The code starts with a while loop that runs indefinitely until a valid number is entered for the portfolio size. 
It prompts the user to enter their portfolio size and attempts to convert the input to a float value using the float() function.
If the conversion is successful (no ValueError is raised), the loop is exited using the break statement.
If the conversion fails (a ValueError is raised), an error message is printed, indicating that a number should be entered.
After exiting the loop, the variable val contains the portfolio size entered by the user.
The code calculates the position size for each stock by dividing the portfolio size (val) by the number of stocks in the result_df DataFrame.

In [109]:
while True:
    try:
        val = float(input("Enter your portfolio size: "))
        break  # Exit the loop if a valid number is entered
    except ValueError:
        print("Please enter a number.")
position_size = val/len(result_df)
for i in range(len(result_df)):
    result_df.loc[i,'Number_of_shares_to_buy'] = math.floor(position_size/float(result_df.loc[i, 'Stock_Price']))

Enter your portfolio size: 1000


The code is responsible for writing the data from the result_df DataFrame to a CSV file named "trades". It uses the csv module to accomplish this task.

Let's break down the code line by line:

with open('trades', 'w') as f:
This line opens the file named "trades" in write mode ('w'). The file will be created if it doesn't exist, and if it exists, its contents will be overwritten. The file object is assigned to the variable f.

writer = csv.writer(f)
This line creates a CSV writer object using the file object f as the output file. The csv.writer() function is used to create the writer.

In [113]:
with open('trades', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(["Ticker", "Stock_Price", "Market_Cap", "Number_of_shares_to_buy"])  # Write header row
    
    for i in range(len(result_df)):
        ticker = result_df.loc[i, "Ticker"]
        stock_price = result_df.loc[i, "Stock_Price"]
        market_cap = result_df.loc[i, "Market_Cap"]
        num_shares_to_buy = result_df.loc[i, "Number_of_shares_to_buy"]
        
        writer.writerow([ticker, stock_price, market_cap, num_shares_to_buy])  # Write data row